In [458]:
# fantasy PL
%reset -f
import pandas as pd
import requests
import json

In [459]:
# api endpoints
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
player_history_url = 'https://fantasy.premierleague.com/api/element-summary/{player_id}/'

In [460]:
# making request
response = requests.get(url)

# check status
if response.status_code == 200:
    print('The API request was successful.')
else:
    print(f'Error: The API request failed with status code {response.status_code}.')

The API request was successful.


In [461]:
# get all players
response = requests.get(url)
players = response.json()['elements']

# print(json.dumps(players, indent=4, sort_keys=True))

In [462]:
# positions mapping
element_types = response.json()['element_types']
position_mapping = {}

for pos in element_types:
    position_mapping[pos['id']] = pos['singular_name']

In [463]:
# extract player info
player_info = []

for player in players:
    player_id = player['id']
    name = player['first_name'] + ' ' + player['second_name']
    position = position_mapping[player['element_type']]
    points = player['total_points']
    bonus = player['bonus']
    value = player['now_cost']/10
    minutes = player['minutes']
    ppg = player['points_per_game']  
    goals = player['goals_scored']
    xg = player['expected_goals']
    xg90 = player['expected_goals_per_90']
    assists = player['assists']
    xa = player['expected_assists']
    xa90 = player['expected_assists_per_90']
    influence = player['influence']
    threat = player['threat']  
    creativity = player['creativity']
    transfers_in = player['transfers_in_event']
    transfers_out = player['transfers_out_event']
  
    player_info.append({
        'name': name,
        'position': position,
        'points': points,
        'points - bonus': int(points) - bonus,
        'xp': float(xa) * 3 + float(xg) * 5,
        'xp90': round((float(xa) * 3 + float(xg) * 5) / (float(minutes)/90) if float(minutes) > 90 else 0, 2),
        'value': value,
        'minutes': minutes,
        'ppg': ppg,
        'goals': goals,
        'xg': xg,
        'xg90': xg90,
        'assists': assists,
        'xa': xa,
        'xa90': xa90,
        'influence': influence,
        'threat': threat,
        'creativity': creativity,
        'transfers_in': transfers_in,
        'transfers_out': transfers_out
    })
    
df = pd.DataFrame(player_info)

In [464]:
# midfielders
filtered_mid = df[(df['position'] == 'Midfielder') & ((df['xg90'] > 0.2) | (df['xa90'] > 0.2)) & (df['minutes'] > 90) ]
sorted_mid = filtered_mid.sort_values(by='xp', ascending=False)

sorted_mid

,name,position,points,points - bonus,xp,xp90,value,minutes,ppg,goals,xg,xg90,assists,xa,xa90,influence,threat,creativity,transfers_in,transfers_out
120,Bryan Mbeumo,Midfielder,32,29,22.86,5.71,6.8,360,8.0,4,4.02,1.00,0,0.92,0.23,120.2,126.0,63.1,24560,17662
455,Bruno Borges Fernandes,Midfielder,16,13,15.07,5.02,8.5,270,5.3,1,2.00,0.67,1,1.69,0.56,108.4,81.0,145.8,10579,9244
619,James Maddison,Midfielder,29,25,13.40,3.58,7.7,337,7.2,2,1.60,0.43,2,1.80,0.48,108.6,98.0,148.2,85662,3441
645,Jarrod Bowen,Midfielder,30,25,13.40,3.35,7.1,360,7.5,3,2.38,0.60,1,0.50,0.12,147.0,177.0,61.6,31687,10305
18,Bukayo Saka,Midfielder,21,17,12.63,4.24,8.7,268,7.0,2,1.71,0.57,0,1.36,0.46,115.4,134.0,139.7,11775,8751
227,Enzo Fernández,Midfielder,5,5,10.64,2.66,5.0,360,1.2,0,1.36,0.34,0,1.28,0.32,42.8,49.0,146.7,2345,7337
292,Abdoulaye Doucouré,Midfielder,12,12,10.43,2.61,5.5,360,3.0,1,1.93,0.48,0,0.26,0.06,30.0,78.0,63.8,374,267
370,Mohamed Salah,Midfielder,15,15,10.07,3.61,12.5,251,5.0,1,1.69,0.61,2,0.54,0.19,86.6,107.0,78.1,4612,16805
156,Kaoru Mitoma,Midfielder,23,20,9.57,2.46,6.6,350,5.8,1,0.81,0.21,3,1.84,0.47,96.4,89.0,143.3,10879,23798
658,Lucas Tolentino Coelho de Lima,Midfielder,15,15,9.48,2.41,6.0,354,3.8,1,1.41,0.36,1,0.81,0.21,107.8,61.0,105.8,3201,1732
